In [ ]:
from pyspark.sql import *
from pyspark import SparkConf, SparkContext, SQLContext

In [ ]:
path = "/Users/haynes/Desktop/independent study/data/Cases_J_2009_Sep_11_2_8982.txt"
lines = sc.textFile(path)
lines.collect()

In [ ]:
subsections = lines.filter(lambda line: len(line.split()) >0 and len(line.split()) <=3)

In [ ]:
subsections.collect()

In [ ]:
subsections_list = []
for s in subsections.collect():
    subsections_list.append(s)
subsections_list

In [ ]:
import re
v1 = 'ABC'
v2 = 'XYZ'
regex = re.compile('{}(.*){}'.format(re.escape(v1), re.escape(v2)))
match = re.search(regex,'Blah blah ABC the string to be retrieved XYZ Blah Blah')
print match.group(1)

In [ ]:
a = "intro"
b = "hello maria"
tup = (a,b)
tup

In [ ]:
f = open(path, 'r')
lines1 = f.readlines()

In [ ]:
final_list = []
final_dict = {}

for l in range(0,len(subsections_list)):
    if l!= len(subsections_list) - 1:
        w1 = subsections_list[l]
        w2 = subsections_list[l + 1]
        regex = re.compile('{}(.*){}'.format(re.escape(w1), re.escape(w2)))
        match = re.search(regex,str(lines1))
    
    else:
        w1 = subsections_list[l]
        regex = re.compile('{}(.*)'.format(re.escape(w1)))
        match = re.search(regex,str(lines1))
        
    txt = match.group(1)
    tup= (w1,txt)
    final_list.append(tup)
    
    final_dict[w1] = txt
    
final_dict.keys()

In [ ]:
col_names = [x[0] for x in final_list]

rdd = sc.parallelize(final_list)
sqlContext.createDataFrame(rdd).collect()
df = sqlContext.createDataFrame(rdd)
df.collect()

In [ ]:
df.write.parquet("sample1.parquet") 

In [ ]:
newDataDF = sqlContext.read.parquet("sample.parquet")       
newDataDF.show() 

In [ ]:
final_list

*** VERSION 2 ***

In [18]:
import os, sys, stat
from stat import *
import re
import pandas as pd
from pyspark.sql import *
from pyspark import SparkConf, SparkContext, SQLContext
import unicodedata
from pyspark.sql.functions import *

In [2]:
def normalize(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            l = lines[i].decode('utf-8')
            lines[i] = unicodedata.normalize("NFKD", l).encode('ascii','ignore')
    
    return lines

In [3]:
def parse(path):
    
    lines = sc.textFile(path)
    lines.collect()
    
    # finding the subsections and saving it in a list subsections_list in lower case
    subsections = lines.filter(lambda line: len(line.split()) >0 and len(line.split()) <=3)
    subsections.collect()
    subsections_list = []
    for s in subsections.collect():
        subsections_list.append(s)
       
    # creating a dictionary subsection as the key and the text as the value
    final_list = []
    final_dict = {}
    
    import unicodedata

    for i in range(len(subsections_list)):
        subsections_list[i] = unicodedata.normalize("NFKD", subsections_list[i]).encode('ascii','ignore')
   
    #f = open(path, 'r')
    #lines1 = f.readlines()
    
    lines1 = normalize(path)
    
    for l in range(0,len(subsections_list)):
        
        # saving the text btween two subsections
        if l!= len(subsections_list) - 1:
            w1 = subsections_list[l]
            w2 = subsections_list[l + 1]
            regex = re.compile('{}(.*){}'.format(re.escape(w1), re.escape(w2)))
            match = re.search(regex,str(lines1))
        
        # saving the last section
        else:
            w1 = subsections_list[l]
            regex = re.compile('{}(.*)'.format(re.escape(w1)))
            match = re.search(regex,str(lines1))
        
        txt = match.group(1)
        tup= (w1,txt)
        final_list.append(tup)
        w1 = w1.replace(' ','_')
        final_dict[w1.lower()] = txt
        
    return final_dict

In [4]:
def mkf2id(full_text_file):
    f2id=dict()
    with open(full_text_file, 'r') as f:
        for l in f:
            tokens = l.strip(' \t\n').split(' ')
            filename = tokens[-1].replace('.tar.gz', '.txt')
            uniq_id = tokens[0].strip(' \t')
            f2id[filename] = uniq_id
    return f2id

In [5]:
def mkdf(path, f2id):
    items = os.listdir(path)
    uniqueKeys = set()
    docList = []
    
    for item in items:
        pathname = os.path.join(path, item)
        mode = os.stat(pathname).st_mode
        if not stat.S_ISREG(mode):
            continue

        if not pathname.endswith('.txt'):
            continue

        print pathname
        d = parse(pathname)
        d['PMID_KEY'] = f2id[item]
        uniqueKeys.update(d.keys())
        docList.append(d)
        print len(docList)

    df = pd.DataFrame(columns=uniqueKeys)
    for d in docList:
        df=df.append(d, ignore_index=True)

    df = df.fillna('') #or ""
    return df

In [7]:
#path = "/Users/haynes/Desktop/independent study/data"
# Changing the path to include the 100 files containing vegf
path = "/Users/haynes/Desktop/independent study/files containg VEGF/fulltext"
path_fulltextid = "/Users/haynes/Desktop/independent study/files containg VEGF/fulltext_id.txt"
f2id=mkf2id(path_fulltextid)
df = mkdf(path, f2id)

/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Am_J_Physiol_Heart_Circ_Physiol_2012_Mar_15_302(6)_H1261-H1273.txt
1
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Arch_Med_Sci_2014_Aug_29_10(4)_837-845.txt
2
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Arthritis_Res_Ther_2010_Feb_11_12(1)_R22.txt
3
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Biochem_Biophys_Res_Commun_2007_Sep_21_361(2-2)_468-473.txt
4
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Biomed_Res_Int_2013_Sep_16_2013_580135.txt
5
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Biomed_Res_Int_2014_Jul_17_2014_107526.txt
6
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/BMC_Biochem_2011_Apr_15_12_15.txt
7
/Users/haynes/Desktop/independent study/files containg VEGF/fulltext/Br_J_Cancer_2002_Mar_18_86(6)_858-863.txt
8
/Users/haynes/Desktop/independent study/files containg 

In [ ]:
sdf = sqlContext.createDataFrame(df)
#sdf.collect()

In [ ]:
sdf.write.parquet("sampleversion3.parquet") 

In [ ]:
newDataDF = sqlContext.read.parquet("sampleversion3.parquet")       
newDataDF.show() 

In [8]:
df.shape

(63, 285)

In [9]:
# Remove a column that has value only in one row and the value is new line alone
for col in df.columns.tolist():
    if (df[df[col]!=''].shape[0] == 1):
        if(df[df[col]!= ''][col] == '\n', '\n', ' '):
            df.drop([col],inplace=True,axis=1)

In [10]:
df.columns.tolist()

['cell_lines',
 'results',
 'conclusions',
 'rna_isolation',
 'treatment',
 'author_contributions',
 'materials_and_methods',
 'patient_characteristics',
 'discussion_and_conclusions',
 'background',
 'animals',
 '1._introduction',
 'immunohistochemistry_(ihc)',
 'summary',
 'methods',
 'competing_interests',
 'patients_and_methods',
 'histology',
 'material_and_methods',
 'immunohistochemistry',
 'supplementary_data',
 'case_report',
 'statistical_analysis',
 'PMID_KEY',
 'supporting_information',
 'cell_culture',
 'adverse_events',
 'introduction',
 'abbreviations',
 '.',
 'quantitative_real-time_pcr',
 "authors'_contributions",
 'future_directions',
 'discussion',
 'conclusion',
 'cell_cultures']

In [11]:
# merging columns 
# conclusion and conclusions
df['conclusion'] = df[['conclusion', 'conclusions']].apply(lambda x: ''.join(x), axis=1)
df.drop(['conclusions'],inplace=True,axis=1)

df['immunohistochemistry'] = df[['immunohistochemistry', 'immunohistochemistry_(ihc)']].apply(lambda x: ''.join(x), axis=1)
df.drop(['immunohistochemistry_(ihc)'],inplace=True,axis=1)

df['introduction'] = df[['introduction', '1._introduction']].apply(lambda x: ''.join(x), axis=1)
df.drop(['1._introduction'],inplace=True,axis=1)

df['author_contributions'] = df[['author_contributions', "authors'_contributions"]].apply(lambda x: ''.join(x), axis=1)
df.drop(["authors'_contributions"],inplace=True,axis=1)

df['cell_culture'] = df[['cell_culture', 'cell_cultures']].apply(lambda x: ''.join(x), axis=1)
df.drop(['cell_cultures'],inplace=True,axis=1)

df['materials_and_methods'] = df[['materials_and_methods', "material_and_methods"]].apply(lambda x: ''.join(x), axis=1)
df.drop(["material_and_methods"],inplace=True,axis=1)

In [12]:
df.shape

(63, 30)

In [15]:
sdf = sqlContext.createDataFrame(df)

In [16]:
sdf.registerTempTable("VEGF")

In [ ]:
# Register the DataFrame as a SQL temporary view
sdf.createOrReplaceTempView("vegf")

sqlDF = spark.sql("SELECT * FROM vegf")
sqlDF.show()

In [20]:
contains_vegf = spark.sql("SELECT count(immunohistochemistry) FROM VEGF where immunohistochemistry like '%VEGF%'")
contains_vegf.show()

#rows_vegf = spark.sql("SELECT (ROUND(length(conclusion))-ROUND(length(regexp_replace(conclusion,'VEGF',''))))/4 AS count_VEGF_conclusion FROM VEGF  where conclusion like '%VEGF%'")
#rows_vegf.show()

conclusion_vegf = spark.sql("SELECT conclusion, (ROUND(length(conclusion))-ROUND(length(regexp_replace(conclusion,'VEGF',''))))/4 AS count_VEGF_conclusion FROM VEGF  where conclusion like '%VEGF%'")
conclusion_vegf.show()


# converting the count into a new column

sdf_new = sdf.withColumn("count_VEGF_conclusion",(length('conclusion')-length(regexp_replace('conclusion','VEGF','')))/4)
#sdf_new[['conclusion','count_VEGF_conclusion']].show(50)

sdf_new = sdf_new.withColumn("count_VEGF_ihc",(length('immunohistochemistry')-length(regexp_replace('immunohistochemistry','VEGF','')))/4)
#sdf_new[['immunohistochemistry','count_VEGF_ihc']].show(63)

# adding the count colums
#newdf = sdf_new.withColumn('total', sum(sdf_new[col] for col in ["count_VEGF_conclusion","count_VEGF_ihc"]))
sdf_new.registerTempTable("VEGF_NEW")
spark.sql("select count_VEGF_conclusion + count_VEGF_ihc from VEGF_NEW").show(63)

+---------------------------+
|count(immunohistochemistry)|
+---------------------------+
|                          3|
+---------------------------+

+--------------------+---------------------+
|          conclusion|count_VEGF_conclusion|
+--------------------+---------------------+
|\n', '\n', 'There...|                  1.0|
|\n', '\n', 'COPD ...|                  7.0|
|\n', '\n', 'Neuro...|                  3.0|
|\n', '\n', 'The a...|                  1.0|
+--------------------+---------------------+

+----------------------------------------+
|(count_VEGF_conclusion + count_VEGF_ihc)|
+----------------------------------------+
|                                     0.0|
|                                     0.0|
|                                     0.0|
|                                     0.0|
|                                     0.0|
|                                     0.0|
|                                     0.0|
|                                     0.0|
|              

In [ ]:
from pyspark.sql.functions import *
#sdf.select(expr("length(introduction)")).collect()

concl_vegf = spark.sql("SELECT (ROUND(length(conclusion))-ROUND(length(regexp_replace(conclusion,'VEGF',''))))/4 AS count_VEGF_conclusion FROM VEGF")

# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
count = concl_vegf.rdd.map(lambda p: p.count_VEGF_conclusion).collect()
conclusion_vegf_count_list = []
for c in count:
    conclusion_vegf_count_list.append(c)

a = Column(conclusion_vegf_count_list) 

In [ ]:
spark.sql("select count(*) from VEGF where immunohistochemistry like '%VEGF%' or conclusion like '%VEGF%' or discussion like '%VEGF%'").show()

In [14]:
df[["introduction","PMID_KEY"]]

,introduction,PMID_KEY
0,,22268107
1,"\n', '\n', 'Acromegaly, a chronic disease caus...",25276172
2,"\n', '\n', 'Low back pain (LBP) is the leading...",20149220
3,,17658463
4,"\n', '\n', 'Despite intensive investigations d...",24151609
5,"\n', '\n', 'Meningiomas are the second most co...",25136551
6,,21496221
7,,11953815
8,,18841159
9,,20571495
